# PACKAGES AND LIBRARIES

# PATH AND LABEL PROCESS

In [ ]:
#GENERAL
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import random
import time
#PATH PROCESS
import os
import os.path
from pathlib import Path
import glob
#IMAGE PROCESS
from PIL import Image
from keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import load_img
import cv2
from keras.applications.vgg16 import preprocess_input, decode_predictions
from keras.preprocessing import image
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
#SCALER & TRANSFORMATION
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split
from keras import regularizers
from sklearn.preprocessing import LabelEncoder
#ACCURACY CONTROL
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, roc_auc_score, roc_curve
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
#OPTIMIZER
from keras.optimizers import RMSprop,Adam,Optimizer,Optimizer, SGD
#MODEL LAYERS
from tensorflow.keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization,MaxPooling2D,BatchNormalization,\
                        Permute, TimeDistributed, Bidirectional,GRU, SimpleRNN, LSTM, GlobalAveragePooling2D, SeparableConv2D, ZeroPadding2D, Convolution2D, ZeroPadding2D
from keras import models
from keras import layers
import tensorflow as tf
from keras.applications import VGG16,VGG19,inception_v3
from keras import backend as K
from keras.utils import plot_model
from keras.models import load_model
#SKLEARN CLASSIFIER
from xgboost import XGBClassifier, XGBRegressor
from lightgbm import LGBMClassifier, LGBMRegressor
from catboost import CatBoostClassifier, CatBoostRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.cross_decomposition import PLSRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import Lasso
from sklearn.linear_model import LassoCV
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import ElasticNetCV
#IGNORING WARNINGS
from warnings import filterwarnings
filterwarnings("ignore",category=DeprecationWarning)
filterwarnings("ignore", category=FutureWarning) 
filterwarnings("ignore", category=UserWarning)

#PLOT TYPE
plt.style.use("classic")

#### PATH

In [8]:
Data_Path = Path("./Dataset")

In [9]:
PNG_Path = list(Data_Path.glob(r"*/*.png"))

#### LABEL

In [10]:
PNG_Labels = list(map(lambda x: os.path.split(os.path.split(x)[0])[1],PNG_Path))

#### TO SERIES

In [11]:
PNG_Path_Series = pd.Series(PNG_Path,name="PNG").astype(str)
PNG_Labels_Series = pd.Series(PNG_Labels,name="CATEGORY")

#### TO DATAFRAME

In [12]:
Main_Data = pd.concat([PNG_Path_Series,PNG_Labels_Series],axis=1)

In [13]:
print(Main_Data.head(-1))

ModuleNotFoundError: No module named 'pandas.io.formats.string'

In [8]:
print(Main_Data["CATEGORY"].value_counts())

#### SHUFFLING

In [9]:
Main_Data = Main_Data.sample(frac=1).reset_index(drop=True)

In [10]:
print(Main_Data.head(-1))

# VISUALIZATION

In [11]:
def general_showing(integer_of_image):

    Example_PNG = Main_Data["PNG"][integer_of_image]
    Reading_IMG = cv2.imread(Example_PNG)
    Transformation_RGB = cv2.cvtColor(Reading_IMG,cv2.COLOR_BGR2RGB)

    plt.xlabel(Transformation_RGB.shape)
    plt.ylabel(Transformation_RGB.size)
    plt.title(Main_Data["CATEGORY"][integer_of_image])

    plt.imshow(Transformation_RGB)


In [12]:
figure = plt.figure(figsize=(7,7))
general_showing(5909)

In [13]:
figure = plt.figure(figsize=(7,7))
general_showing(3)

In [14]:
figure = plt.figure(figsize=(7,7))
general_showing(8976)

In [15]:
figure,axis = plt.subplots(nrows=5,ncols=5,figsize=(14,14))

for indexing,run_axes in enumerate(axis.flat):
    Reading_IMG = cv2.imread(Main_Data["PNG"][indexing])
    Transformation_IMG = cv2.cvtColor(Reading_IMG,cv2.COLOR_BGR2RGB)
    
    run_axes.set_xlabel(Transformation_IMG.shape)
    run_axes.set_ylabel(Transformation_IMG.size)
    run_axes.set_title(Main_Data["CATEGORY"][indexing])
    
    run_axes.imshow(Transformation_IMG)
    
plt.tight_layout()
plt.show()

# SPLITTING TEST AND TRAIN DATA

In [16]:
Train_Data,Test_Data = train_test_split(Main_Data,train_size=0.9,shuffle=True,random_state=42)

In [17]:
print(Train_Data.shape)

In [18]:
print(Test_Data.shape)

# IMAGE GENERATOR

#### STRUCTURE

In [19]:
Train_IMG_Generator = ImageDataGenerator(rescale=1./255,
                                        rotation_range=25,
                                        shear_range=0.5,
                                        zoom_range=0.5,
                                        width_shift_range=0.2,
                                        height_shift_range=0.2,
                                        horizontal_flip=True,
                                        fill_mode="nearest",
                                        validation_split=0.1)

In [20]:
Test_IMG_Generator = ImageDataGenerator(rescale=1./255)

#### APPLY

In [21]:
Train_IMG_Set = Train_IMG_Generator.flow_from_dataframe(dataframe=Train_Data,
                                                       x_col="PNG",
                                                       y_col="CATEGORY",
                                                       color_mode="rgb",
                                                       class_mode="categorical",
                                                       target_size=(128,128),
                                                       subset="training")

In [22]:
Validation_IMG_Set = Train_IMG_Generator.flow_from_dataframe(dataframe=Train_Data,
                                                       x_col="PNG",
                                                       y_col="CATEGORY",
                                                       color_mode="rgb",
                                                       class_mode="categorical",
                                                       target_size=(128,128),
                                                       subset="validation")

In [23]:
Test_IMG_Set = Test_IMG_Generator.flow_from_dataframe(dataframe=Test_Data,
                                                       x_col="PNG",
                                                       y_col="CATEGORY",
                                                       color_mode="rgb",
                                                       class_mode="categorical",
                                                       target_size=(128,128),
                                                       shuffle=False)

In [24]:
print("TRAIN: ")
print(Train_IMG_Set.class_indices)
print(Train_IMG_Set.classes[0:5])
print(Train_IMG_Set.image_shape)
print("---"*20)
print("VALIDATION: ")
print(Validation_IMG_Set.class_indices)
print(Validation_IMG_Set.classes[0:5])
print(Validation_IMG_Set.image_shape)
print("---"*20)
print("TEST: ")
print(Test_IMG_Set.class_indices)
print(Test_IMG_Set.classes[0:5])
print(Test_IMG_Set.image_shape)

#### HOW TO LOOK BY GENERATOR

In [25]:
Example_Image = Train_Data["PNG"][422]
Reading_Image = cv2.imread(Example_Image)
Transformation_Image = cv2.cvtColor(Reading_Image,cv2.COLOR_BGR2RGB)
Array_Image = image.img_to_array(Transformation_Image)
Array_Image = Array_Image.reshape((1,)+Array_Image.shape)

i = 0

for batch in Train_IMG_Generator.flow(Array_Image,batch_size=32):
    plt.figure(i)
    
    Image = plt.imshow(image.img_to_array(batch[0]))
    
    i += 1
    
    if i % 4 == 0:
        break

plt.show()

# MODEL

In [ ]:
Model = Sequential()

#
Model.add(Conv2D(32,(3,3),activation="relu",input_shape=(128,128,3)))
Model.add(BatchNormalization())
Model.add(MaxPooling2D((2,2)))

#
Model.add(Conv2D(64,(3,3),padding="same",activation="relu"))
Model.add(Dropout(0.3))
Model.add(MaxPooling2D((2,2)))

Model.add(Conv2D(128,(3,3),padding="same",activation="relu"))
Model.add(Dropout(0.3))
Model.add(MaxPooling2D((2,2)))

Model.add(Conv2D(128,(3,3),padding="same",activation="relu"))
Model.add(Dropout(0.3))
Model.add(MaxPooling2D((2,2)))

#
Model.add(Flatten())
Model.add(Dense(256,activation="relu"))
Model.add(Dropout(0.5))

#
Model.add(Dense(3,activation="softmax"))

In [ ]:
Early_Stop = tf.keras.callbacks.EarlyStopping(monitor="loss",patience=3,mode="min")

In [ ]:
Model.compile(optimizer="adam",loss="categorical_crossentropy",metrics=["accuracy"])

In [ ]:
print(Model.summary())

In [ ]:
CNN_Sep_Model = Model.fit(Train_IMG_Set,validation_data=Validation_IMG_Set,callbacks=Early_Stop,epochs=50)

#### CHECKING

In [ ]:
Grap_Data = pd.DataFrame(CNN_Sep_Model.history)
figure = plt.figure(figsize=(10,10))

Grap_Data.plot()

In [ ]:
plt.plot(CNN_Sep_Model.history["accuracy"])
plt.plot(CNN_Sep_Model.history["val_accuracy"])
plt.ylabel("ACCURACY")
plt.legend()
plt.show()

In [ ]:
plt.plot(CNN_Sep_Model.history["loss"])
plt.plot(CNN_Sep_Model.history["val_loss"])
plt.ylabel("LOSS")
plt.legend()
plt.show()

#### SAVING

In [ ]:
Model.save("main_model.h5")
Model.save_weights("main_model_weight.h5")

#### PREDICTION

In [ ]:
Model_Results = Model.evaluate(Test_IMG_Set)
print("LOSS:  " + "%.4f" % Model_Results[0])
print("ACCURACY:  " + "%.4f" % Model_Results[1])

In [ ]:
Prediction_Number = Model.predict(Test_IMG_Set)

In [ ]:
print(Prediction_Number[0:10])

In [ ]:
Prediction = Prediction_Number.argmax(axis=-1)

In [ ]:
print(Prediction[0:10])

* mask_weared_incorrect: 0
* with_mask: 1
* without_mask: 2

In [ ]:
fig, axes = plt.subplots(nrows=5,
                         ncols=5,
                         figsize=(20, 20),
                        subplot_kw={'xticks': [], 'yticks': []})

for i, ax in enumerate(axes.flat):
    ax.imshow(plt.imread(Test_Data["PNG"].iloc[i]))
    ax.set_title(f"PREDICTION:{Prediction[i]}")
    ax.set_xlabel(Test_Data["CATEGORY"].iloc[i])
plt.tight_layout()
plt.show()

#### TARGET PREDICT

In [ ]:
CNN_Main_Model = load_model("./main_model.h5")

##### NO MASK

In [ ]:
Non_Seen_Image = "../input/face-mask-detection/Dataset/without_mask/1023.png"

In [ ]:
Reading_IMG = cv2.imread(Non_Seen_Image)
Transformation_RGB = cv2.cvtColor(Reading_IMG,cv2.COLOR_BGR2RGB)

plt.xlabel(Transformation_RGB.shape)
plt.ylabel(Transformation_RGB.size)

plt.imshow(Transformation_RGB)

In [ ]:
img = load_img(Non_Seen_Image, target_size=(128, 128))  # Loading image
img = image.img_to_array(img)  # Transforming image to array
img = img / 255  # Normalizing Image
img = np.expand_dims(img, axis=0)

print(img.shape)

In [ ]:
Non_Seen_Predict = Model.predict(img)

In [ ]:
Non_Seen_Predict = Non_Seen_Predict.argmax(axis=-1)
print(Non_Seen_Predict)

In [ ]:
img_x = load_img(Non_Seen_Image, target_size=(128, 128))  # Loading image
img_x = image.img_to_array(img_x)  # Transforming image to array
img_x = img_x / 255  # Normalizing Image
img_x = np.expand_dims(img_x, axis=0)

In [ ]:
Non_Seen_Predict_x = Model.predict(img_x)[0]

In [ ]:
(startX, startY, prb) = Non_Seen_Predict_x

In [ ]:
print(f"{startX} x {startY} x {prb}")

In [ ]:
(h,w) = Transformation_RGB.shape[:2]

In [ ]:
print(f"{h} x {w}")

In [ ]:
startX = int(startX * w)
startY = int(startY * h)
prb = int(prb * w)

In [ ]:
print(f"{startX} x {startY} x {prb}")

In [ ]:
Reading_cv = cv2.imread(Non_Seen_Image)

##### MASK

In [ ]:
Mask_Image = "../input/face-mask-detection/Dataset/with_mask/1009.png"

In [ ]:
Reading_IMG = cv2.imread(Mask_Image)
Transformation_RGB = cv2.cvtColor(Reading_IMG,cv2.COLOR_BGR2RGB)

plt.xlabel(Transformation_RGB.shape)
plt.ylabel(Transformation_RGB.size)

plt.imshow(Transformation_RGB)

In [ ]:
img = load_img(Mask_Image, target_size=(128, 128))  # Loading image
img = image.img_to_array(img)  # Transforming image to array
img = img / 255  # Normalizing Image
img = np.expand_dims(img, axis=0)

print(img.shape)

In [ ]:
Non_Seen_Predict_Mask = Model.predict(img)

In [ ]:
Non_Seen_Predict_Mask = Non_Seen_Predict_Mask.argmax(axis=-1)
print(Non_Seen_Predict_Mask)

In [ ]:
img_x = load_img(Mask_Image, target_size=(128, 128))  # Loading image
img_x = image.img_to_array(img_x)  # Transforming image to array
img_x = img_x / 255  # Normalizing Image
img_x = np.expand_dims(img_x, axis=0)

In [ ]:
Non_Seen_Predict_x_mask = Model.predict(img_x)[0]

In [ ]:
(startX, startY, prb) = Non_Seen_Predict_x_mask

In [ ]:
print(f"{startX} x {startY} x {prb}")

In [ ]:
(h,w) = Reading_IMG.shape[:2]

In [ ]:
half_h = int(w / 2)
half_w = int(h / 2)

In [ ]:
print(f"{half_h} x {half_w}")

In [ ]:
print(f"{h} x {w}")

In [ ]:
startX = int(startX * w)
startY = int(startY * h)
prb = int(prb * w)

In [ ]:
print(f"{startX} x {startY} x {prb}")